# RNA-seq Data Exploration and Analysis
## GSE313799: Effect of chronic mechanical compression on human induced neurons

This notebook provides an interactive walkthrough of the RNA-seq analysis pipeline.

In [ ]:
import sys
sys.path.insert(0, '../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('Set2')

## 1. Load and Explore Raw Data

In [ ]:
from preprocessing.data_loader import RNAseqDataLoader

project_root = Path('..')
counts_file = project_root / 'data/raw/GSE313799_counts_matrix_iN.txt'
db_path = project_root / 'db/rnaseq_analysis.db'

loader = RNAseqDataLoader(str(counts_file), str(db_path))
counts = loader.load_counts()
metadata = loader.parse_sample_metadata()

print(f"Counts matrix shape: {counts.shape}")
print(f"\nFirst few genes:")
counts.head()

In [ ]:
# Sample metadata
print("Sample Metadata:")
metadata

In [ ]:
# QC metrics
qc = loader.calculate_qc_metrics()
qc

## 2. Library Size Distribution

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Library sizes
lib_sizes = counts.sum(axis=0)
colors = metadata['condition'].map({'compress': 'coral', 'control': 'steelblue'})

ax1 = axes[0]
bars = ax1.bar(range(len(lib_sizes)), lib_sizes.values / 1e6, color=colors.values)
ax1.set_xlabel('Sample')
ax1.set_ylabel('Library Size (millions)')
ax1.set_title('Library Size per Sample')
ax1.axhline(lib_sizes.median() / 1e6, color='red', linestyle='--', label='Median')
ax1.legend()

# Detected genes
ax2 = axes[1]
detected = (counts > 0).sum(axis=0)
ax2.bar(range(len(detected)), detected.values, color=colors.values)
ax2.set_xlabel('Sample')
ax2.set_ylabel('Number of Detected Genes')
ax2.set_title('Detected Genes per Sample')

plt.tight_layout()
plt.show()

## 3. Filter and Normalize

In [ ]:
from preprocessing.normalization import RNAseqNormalizer

# Filter low-count genes
keep = (counts >= 10).sum(axis=1) >= 3
filtered = counts.loc[keep]
print(f"Genes before filtering: {counts.shape[0]}")
print(f"Genes after filtering: {filtered.shape[0]}")

# Normalize
normalizer = RNAseqNormalizer(filtered)
log_cpm = normalizer.cpm(log=True)
tmm_norm = normalizer.tmm_normalize(log=True)

In [ ]:
# Compare distributions before and after normalization
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Raw counts (log scale)
axes[0].boxplot([np.log2(filtered[col] + 1) for col in filtered.columns])
axes[0].set_title('Raw Counts (log2)')
axes[0].set_xticklabels([])

# Log-CPM
axes[1].boxplot([log_cpm[col] for col in log_cpm.columns])
axes[1].set_title('Log-CPM Normalized')
axes[1].set_xticklabels([])

# TMM normalized
axes[2].boxplot([tmm_norm[col] for col in tmm_norm.columns])
axes[2].set_title('TMM Normalized (log)')
axes[2].set_xticklabels([])

plt.tight_layout()
plt.show()

## 4. PCA Analysis (Before Batch Correction)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# PCA on log-CPM data
X = log_cpm.T.values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=10)
scores = pca.fit_transform(X_scaled)

pca_df = pd.DataFrame(scores[:, :3], columns=['PC1', 'PC2', 'PC3'], index=log_cpm.columns)
pca_df = pca_df.join(metadata.set_index('sample_id'))

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Color by condition
for cond in pca_df['condition'].unique():
    mask = pca_df['condition'] == cond
    axes[0].scatter(pca_df.loc[mask, 'PC1'], pca_df.loc[mask, 'PC2'], 
                   label=cond, s=100, alpha=0.7)
axes[0].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)')
axes[0].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)')
axes[0].set_title('PCA: Color by Condition')
axes[0].legend()

# Color by batch
for batch in pca_df['batch'].unique():
    mask = pca_df['batch'] == batch
    axes[1].scatter(pca_df.loc[mask, 'PC1'], pca_df.loc[mask, 'PC2'], 
                   label=batch, s=100, alpha=0.7)
axes[1].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)')
axes[1].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)')
axes[1].set_title('PCA: Color by Batch (Sequencing Date)')
axes[1].legend()

plt.tight_layout()
plt.show()

## 5. Batch Correction

In [ ]:
from batch_correction.combat import ComBat

batch = metadata.set_index('sample_id')['batch']

combat = ComBat(
    data=log_cpm,
    batch=batch,
    parametric=True
)
corrected = combat.fit_transform()
print("Batch correction complete")

In [ ]:
# PCA after batch correction
X_corr = corrected.T.values
X_corr_scaled = scaler.fit_transform(X_corr)

pca_corr = PCA(n_components=10)
scores_corr = pca_corr.fit_transform(X_corr_scaled)

pca_corr_df = pd.DataFrame(scores_corr[:, :3], columns=['PC1', 'PC2', 'PC3'], index=corrected.columns)
pca_corr_df = pca_corr_df.join(metadata.set_index('sample_id'))

# Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for cond in pca_corr_df['condition'].unique():
    mask = pca_corr_df['condition'] == cond
    axes[0].scatter(pca_corr_df.loc[mask, 'PC1'], pca_corr_df.loc[mask, 'PC2'], 
                   label=cond, s=100, alpha=0.7)
axes[0].set_xlabel(f'PC1 ({pca_corr.explained_variance_ratio_[0]*100:.1f}%)')
axes[0].set_ylabel(f'PC2 ({pca_corr.explained_variance_ratio_[1]*100:.1f}%)')
axes[0].set_title('After Batch Correction: Color by Condition')
axes[0].legend()

for batch in pca_corr_df['batch'].unique():
    mask = pca_corr_df['batch'] == batch
    axes[1].scatter(pca_corr_df.loc[mask, 'PC1'], pca_corr_df.loc[mask, 'PC2'], 
                   label=batch, s=100, alpha=0.7)
axes[1].set_xlabel(f'PC1 ({pca_corr.explained_variance_ratio_[0]*100:.1f}%)')
axes[1].set_ylabel(f'PC2 ({pca_corr.explained_variance_ratio_[1]*100:.1f}%)')
axes[1].set_title('After Batch Correction: Color by Batch')
axes[1].legend()

plt.tight_layout()
plt.show()

## 6. Differential Expression Analysis

In [ ]:
from de_analysis.differential_expression import DEAnalysis, filter_significant

de = DEAnalysis(
    filtered,
    metadata.set_index('sample_id'),
    condition_col='condition'
)

de_results = de.run_deseq2_like(contrast=('compress', 'control'))
print(f"Total genes tested: {len(de_results)}")
print(f"Significant genes (padj < 0.05): {(de_results['padj'] < 0.05).sum()}")

In [ ]:
# Volcano plot
fig, ax = plt.subplots(figsize=(10, 8))

# Add -log10(pvalue) column
de_results['neg_log10_pvalue'] = -np.log10(de_results['pvalue'] + 1e-300)

# Color by significance
colors = np.where(
    (de_results['padj'] < 0.05) & (de_results['log2FoldChange'] > 1), 'red',
    np.where(
        (de_results['padj'] < 0.05) & (de_results['log2FoldChange'] < -1), 'blue',
        'gray'
    )
)

ax.scatter(de_results['log2FoldChange'], de_results['neg_log10_pvalue'], 
          c=colors, alpha=0.5, s=10)

ax.axhline(-np.log10(0.05), color='gray', linestyle='--', alpha=0.5)
ax.axvline(1, color='gray', linestyle='--', alpha=0.5)
ax.axvline(-1, color='gray', linestyle='--', alpha=0.5)

ax.set_xlabel('Log2 Fold Change')
ax.set_ylabel('-Log10(p-value)')
ax.set_title('Volcano Plot: Compress vs Control')

# Annotate top genes
top_genes = de_results.nsmallest(10, 'padj')
for _, row in top_genes.iterrows():
    ax.annotate(row['gene'], (row['log2FoldChange'], row['neg_log10_pvalue']),
               fontsize=8, alpha=0.7)

plt.tight_layout()
plt.show()

In [ ]:
# Top DE genes
print("Top 20 Differentially Expressed Genes:")
de_results.head(20)[['gene', 'baseMean', 'log2FoldChange', 'pvalue', 'padj']]

## 7. Feature Selection and ML

In [ ]:
from feature_engineering.feature_selection import FeatureSelector, prepare_ml_data
from ml_models.classifiers import RNAseqClassifier

# Prepare data
X, y = prepare_ml_data(corrected, metadata.set_index('sample_id'), 'condition')
print(f"Feature matrix shape: {X.shape}")
print(f"Label distribution: {y.value_counts().to_dict()}")

In [ ]:
# Feature selection
selector = FeatureSelector(X, y)
selector.variance_filter(top_n=500)

X_selected, y_selected = selector.get_selected_data('variance')
print(f"Selected features: {X_selected.shape[1]}")

In [ ]:
# Train models
classifier = RNAseqClassifier(X_selected, y_selected, test_size=0.2)
results = classifier.train_all_models(tune_hyperparameters=True, cv_folds=5)

print("\nModel Comparison:")
results

In [ ]:
# Feature importance
importance = classifier.get_feature_importance('RandomForest')

# Plot top 20
fig, ax = plt.subplots(figsize=(10, 8))
top20 = importance.head(20)
ax.barh(range(len(top20)), top20['importance'].values)
ax.set_yticks(range(len(top20)))
ax.set_yticklabels(top20['feature'].values)
ax.invert_yaxis()
ax.set_xlabel('Feature Importance')
ax.set_title('Top 20 Most Important Features (Random Forest)')
plt.tight_layout()
plt.show()

## 8. Model Evaluation

In [ ]:
from validation.model_evaluation import ModelEvaluator
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Create model
model = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

# Evaluate
evaluator = ModelEvaluator(model, X_selected, y_selected)

# Cross-validation
cv_results = evaluator.cross_validation('stratified', n_splits=5)
print(f"Cross-Validation Accuracy: {cv_results['mean_score']:.4f} +/- {cv_results['std_score']:.4f}")
print(f"95% CI: [{cv_results['ci_95_lower']:.4f}, {cv_results['ci_95_upper']:.4f}]")

In [ ]:
# Permutation test
perm_results = evaluator.permutation_test(n_permutations=100)
print(f"\nPermutation Test:")
print(f"Actual Score: {perm_results['score']:.4f}")
print(f"Permutation Scores: {perm_results['permutation_scores_mean']:.4f} +/- {perm_results['permutation_scores_std']:.4f}")
print(f"P-value: {perm_results['pvalue']:.4f}")
print(f"Statistically Significant: {perm_results['significant']}")

## Summary

This notebook demonstrated the complete RNA-seq analysis pipeline:

1. **Data Loading**: Loaded counts matrix with 49,162 genes x 24 samples
2. **Quality Control**: Assessed library sizes and detected genes
3. **Filtering**: Removed low-count genes
4. **Normalization**: Applied CPM and TMM normalization
5. **Batch Correction**: Used ComBat to remove sequencing batch effects
6. **DE Analysis**: Identified differentially expressed genes between compress and control conditions
7. **Feature Selection**: Selected top variance genes for ML
8. **ML Models**: Trained and compared multiple classifiers
9. **Validation**: Evaluated model performance with cross-validation and permutation tests